### preprocess the data

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns
# import scipy.stats
# from tqdm import tqdm

# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import Ridge
# from sklearn.metrics import r2_score, mean_squared_error
# import itertools
# from scipy.optimize import minimize
# from functools import partial
# import random, os
# from astropy.stats import sigma_clip

In [ ]:
# test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
#                            index_col='planet_id')
# axis_info = pd.read_parquet('/kaggle/input/ariel-data-challenge-2024/axis_info.parquet')

In [ ]:
# def apply_linear_corr(linear_corr,clean_signal):
#     linear_corr = np.flip(linear_corr, axis=0)
#     for x, y in itertools.product(
#                 range(clean_signal.shape[1]), range(clean_signal.shape[2])
#             ):
#         poli = np.poly1d(linear_corr[:, x, y])
#         clean_signal[:, x, y] = poli(clean_signal[:, x, y])
#     return clean_signal

# def clean_dark(signal, dark, dt):
#     dark = np.tile(dark, (signal.shape[0], 1, 1))
#     signal -= dark* dt[:, np.newaxis, np.newaxis]
#     return signal

# def preproc(dataset, adc_info, sensor, binning = 15):
#     cut_inf, cut_sup = 39, 321
#     sensor_sizes_dict = {"AIRS-CH0":[[11250, 32, 356], [1, 32, cut_sup-cut_inf]], "FGS1":[[135000, 32, 32], [1, 32, 32]]}
#     binned_dict = {"AIRS-CH0":[11250 // binning // 2, 282], "FGS1":[135000 // binning // 2]}
#     linear_corr_dict = {"AIRS-CH0":(6, 32, 356), "FGS1":(6, 32, 32)}
#     planet_ids = adc_info.index
    
#     feats = []
#     for i, planet_id in tqdm(list(enumerate(planet_ids))):
#         signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/{planet_id}/{sensor}_signal.parquet').to_numpy()
#         dark_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dark.parquet', engine='pyarrow').to_numpy()
#         dead_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dead.parquet', engine='pyarrow').to_numpy()
#         flat_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/flat.parquet', engine='pyarrow').to_numpy()
#         linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/linear_corr.parquet').values.astype(np.float64).reshape(linear_corr_dict[sensor])

#         signal = signal.reshape(sensor_sizes_dict[sensor][0]) 
#         gain = adc_info[f'{sensor}_adc_gain'].values[i]
#         offset = adc_info[f'{sensor}_adc_offset'].values[i]
#         signal = signal / gain + offset
        
#         hot = sigma_clip(
#             dark_frame, sigma=5, maxiters=5
#         ).mask
        
#         if sensor != "FGS1":
#             signal = signal[:, :, cut_inf:cut_sup] #11250 * 32 * 282
#             #dt = axis_info['AIRS-CH0-integration_time'].dropna().values
#             dt = np.ones(len(signal))*0.1 
#             dt[1::2] += 4.5 #@bilzard idea
#             linear_corr = linear_corr[:, :, cut_inf:cut_sup]
#             dark_frame = dark_frame[:, cut_inf:cut_sup]
#             dead_frame = dead_frame[:, cut_inf:cut_sup]
#             flat_frame = flat_frame[:, cut_inf:cut_sup]
#             hot = hot[:, cut_inf:cut_sup]
#         else:
#             dt = np.ones(len(signal))*0.1
#             dt[1::2] += 0.1
            
#         signal = signal.clip(0) #@graySnow idea
#         linear_corr_signal = apply_linear_corr(linear_corr, signal)
#         signal = clean_dark(linear_corr_signal, dark_frame, dt)
        
#         flat = flat_frame.reshape(sensor_sizes_dict[sensor][1])
#         flat[dead_frame.reshape(sensor_sizes_dict[sensor][1])] = np.nan
#         flat[hot.reshape(sensor_sizes_dict[sensor][1])] = np.nan
#         signal = signal / flat
        
#         if sensor == "FGS1":
#             signal = signal.reshape((sensor_sizes_dict[sensor][0][0], sensor_sizes_dict[sensor][0][1]*sensor_sizes_dict[sensor][0][2]))
        
#         mean_signal = np.nanmean(signal, axis=1) # mean over the 32*32(FGS1) or 32(CH0) pixels
#         cds_signal = (mean_signal[1::2] - mean_signal[0::2])
        
#         binned = np.zeros((binned_dict[sensor]))
#         for j in range(cds_signal.shape[0] // binning):
#             binned[j] = cds_signal[j*binning:j*binning+binning].mean(axis=0)
                   
#         if sensor == "FGS1":
#             binned = binned.reshape((binned.shape[0],1))
            
#         feats.append(binned)
        
#     return np.stack(feats)
    
# pre_train = np.concatenate([preproc('test', test_adc_info, "FGS1", 30*12), preproc('test', test_adc_info, "AIRS-CH0", 30)], axis=2)

### fit polynoms for each sample

In [ ]:
# def phase_detector(signal):
#     phase1, phase2 = None, None
#     best_drop = 0
#     for i in range(50//2,150//2):        
#         t1 = signal[i:i+20//2].max() - signal[i:i+20//2].min()
#         if t1 > best_drop:
#             phase1 = i+(20+5)//2
#             best_drop = t1
    
#     best_drop = 0
#     for i in range(200//2,250//2):
#         t1 = signal[i:i+20//2].max() - signal[i:i+20//2].min()
#         if t1 > best_drop:
#             phase2 = i-5//2
#             best_drop = t1
    
#     return phase1, phase2

# def try_s(signal, p1, p2, deg, s):
#     out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
#     x, y = out, signal[out].tolist()
#     x = x + list(range(p1,p2))

#     y = y + (signal[p1:p2] * (1 + s[0])).tolist()
#     z = np.polyfit(x, y, deg)
#     p = np.poly1d(z)
#     q = np.abs(p(x) - y).mean()

#     if s < 1e-4:
#         return q + 1e3

#     return q
    
# def calibrate_signal(signal):
#     p1,p2 = phase_detector(signal)

#     best_deg, best_score = 1, 1e12
#     for deg in range(1, 6):
#         f = partial(try_s, signal, p1, p2, deg)
#         r = minimize(f, [0.001], method = 'Nelder-Mead')
#         s = r.x[0]

#         out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
#         x, y = out, signal[out].tolist()
#         x = x + list(range(p1,p2))
#         y = y + (signal[p1:p2] * (1 + s)).tolist()
    
#         z = np.polyfit(x, y, deg)
#         p = np.poly1d(z)
#         q = np.abs(p(x) - y).mean()
        
#         if q < best_score:
#             best_score = q
#             best_deg = deg
        
#         print(deg, q)
            
#     z = np.polyfit(x, y, best_deg)
#     p = np.poly1d(z)

#     return s, x, y, p(x)

# def calibrate_train(signal):
#     p1,p2 = phase_detector(signal)
    
#     best_deg, best_score = 1, 1e12
#     for deg in range(1, 6):
#         f = partial(try_s, signal, p1, p2, deg)
#         r = minimize(f, [0.001], method = 'Nelder-Mead')
#         s = r.x[0]

#         out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
#         x, y = out, signal[out].tolist()
#         x = x + list(range(p1,p2))
#         y = y + (signal[p1:p2] * (1 + s)).tolist()
    
#         z = np.polyfit(x, y, deg)
#         p = np.poly1d(z)
#         q = np.abs(p(x) - y).mean()
        
#         if q < best_score:
#             best_score = q
#             best_deg = deg
            
#     z = np.polyfit(x, y, best_deg)
#     p = np.poly1d(z)
    
#     return s, p(np.arange(signal.shape[0])), p1, p2


# train = pre_train.copy()
# all_s = []
# for i in range(len(test_adc_info)):
#     signal = train[i,:,1:].mean(axis=1)
#     s, p, p1, p2 = calibrate_train(pre_train[i,:,1:].mean(axis=1))
#     all_s.append(s)
        
# #copy answer 283 times because we predict mean value
# train_s = np.repeat(np.array(all_s), 283).reshape((len(all_s), 283))        
# train_sigma = np.ones_like(train_s) * 0.00016

Probably we can accurately estimate sigma from train

In [ ]:
# n = 0
# s, x, y, y_new = calibrate_signal(pre_train[n,:,1:].mean(axis=1))
# plt.scatter(x,y)
# plt.scatter(x,y_new)

I call the orange line "starline". This is probably what we would see if the planet weren't in the way.

### Making submission

In [ ]:
# ss = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv')

# preds = train_s.clip(0)
# sigmas = train_sigma
# submission = pd.DataFrame(np.concatenate([preds,sigmas], axis=1), columns=ss.columns[1:])
# submission.index = test_adc_info.index
# submission.to_csv('submission_1.csv')

In [ ]:
# submission

## [ariel_local_purple_hat_inference](https://www.kaggle.com/code/sergeifironov/ariel-local-purple-hat-inference)

### [Sergei Fironov](https://www.kaggle.com/sergeifironov)

## [ariel_only_correlation LB: 0.515](https://www.kaggle.com/code/jaejohn/ariel-only-correlation-lb-0-515)

### [G John Rao](https://www.kaggle.com/jaejohn)

### preface 
I was inspired by @AmbrosM idea https://www.kaggle.com/competitions/ariel-data-challenge-2024/discussion/530152#2969648

The idea of ​​this approach is to predict the average answer for each test sample. Instead of building models, we will search for the correct answer experimentally. We will select for each spectrum such a multiplier that the transit part multiplied by it will "line up" with the other points. The line can be either a straight line or a polynomial up to the 5th degree. For selection, we will use the Nelder-Mead method. The found multiplication factor minus one is our answer.

There are some changes in data preparation here.
* dt for dark calibration changed in favor https://www.kaggle.com/code/gordonyip/update-calibrating-and-binning-astronomical-data/comments#2964759
* signal clipped to zero due this https://www.kaggle.com/competitions/ariel-data-challenge-2024/discussion/530247#2970709
* I masked hot and dead pixels with NaN in flat and averaging through spatial dimension.

### **upd for version 2**: it seems I forgot to remove **5x sigma** copied from somewhere. I wonder how this will affect the score.

#### preprocess the data

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns
# import scipy.stats
# from tqdm import tqdm

# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import Ridge
# from sklearn.metrics import r2_score, mean_squared_error
# import itertools
# from scipy.optimize import minimize
# from functools import partial
# import random, os
# from astropy.stats import sigma_clip

In [ ]:
# test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
#                            index_col='planet_id')
# axis_info = pd.read_parquet('/kaggle/input/ariel-data-challenge-2024/axis_info.parquet')

In [ ]:
# def apply_linear_corr(linear_corr,clean_signal):
#     linear_corr = np.flip(linear_corr, axis=0)
#     for x, y in itertools.product(
#                 range(clean_signal.shape[1]), range(clean_signal.shape[2])
#             ):
#         poli = np.poly1d(linear_corr[:, x, y])
#         clean_signal[:, x, y] = poli(clean_signal[:, x, y])
#     return clean_signal

# def clean_dark(signal, dark, dt):
#     dark = np.tile(dark, (signal.shape[0], 1, 1))
#     signal -= dark* dt[:, np.newaxis, np.newaxis]
#     return signal

# def preproc(dataset, adc_info, sensor, binning = 15):
#     cut_inf, cut_sup = 39, 321
#     sensor_sizes_dict = {"AIRS-CH0":[[11250, 32, 356], [1, 32, cut_sup-cut_inf]], "FGS1":[[135000, 32, 32], [1, 32, 32]]}
#     binned_dict = {"AIRS-CH0":[11250 // binning // 2, 282], "FGS1":[135000 // binning // 2]}
#     linear_corr_dict = {"AIRS-CH0":(6, 32, 356), "FGS1":(6, 32, 32)}
#     planet_ids = adc_info.index
    
#     feats = []
#     for i, planet_id in tqdm(list(enumerate(planet_ids))):
#         signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/{planet_id}/{sensor}_signal.parquet').to_numpy()
#         dark_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dark.parquet', engine='pyarrow').to_numpy()
#         dead_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dead.parquet', engine='pyarrow').to_numpy()
#         flat_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/flat.parquet', engine='pyarrow').to_numpy()
#         linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/linear_corr.parquet').values.astype(np.float64).reshape(linear_corr_dict[sensor])

#         signal = signal.reshape(sensor_sizes_dict[sensor][0]) 
#         gain = adc_info[f'{sensor}_adc_gain'].values[i]
#         offset = adc_info[f'{sensor}_adc_offset'].values[i]
#         signal = signal / gain + offset
        
#         hot = sigma_clip(
#             dark_frame, sigma=5, maxiters=5
#         ).mask
        
#         if sensor != "FGS1":
#             signal = signal[:, :, cut_inf:cut_sup] #11250 * 32 * 282
#             #dt = axis_info['AIRS-CH0-integration_time'].dropna().values
#             dt = np.ones(len(signal))*0.1 
#             dt[1::2] += 4.5 #@bilzard idea
#             linear_corr = linear_corr[:, :, cut_inf:cut_sup]
#             dark_frame = dark_frame[:, cut_inf:cut_sup]
#             dead_frame = dead_frame[:, cut_inf:cut_sup]
#             flat_frame = flat_frame[:, cut_inf:cut_sup]
#             hot = hot[:, cut_inf:cut_sup]
#         else:
#             dt = np.ones(len(signal))*0.1
#             dt[1::2] += 0.1
            
#         signal = signal.clip(0) #@graySnow idea
#         linear_corr_signal = apply_linear_corr(linear_corr, signal)
#         signal = clean_dark(linear_corr_signal, dark_frame, dt)
        
#         flat = flat_frame.reshape(sensor_sizes_dict[sensor][1])
#         flat[dead_frame.reshape(sensor_sizes_dict[sensor][1])] = np.nan
#         flat[hot.reshape(sensor_sizes_dict[sensor][1])] = np.nan
#         signal = signal / flat
        
#         if sensor == "FGS1":
#             signal = signal.reshape((sensor_sizes_dict[sensor][0][0], sensor_sizes_dict[sensor][0][1]*sensor_sizes_dict[sensor][0][2]))
        
#         mean_signal = np.nanmean(signal, axis=1) # mean over the 32*32(FGS1) or 32(CH0) pixels
#         cds_signal = (mean_signal[1::2] - mean_signal[0::2])
        
#         binned = np.zeros((binned_dict[sensor]))
#         for j in range(cds_signal.shape[0] // binning):
#             binned[j] = cds_signal[j*binning:j*binning+binning].mean(axis=0)
                   
#         if sensor == "FGS1":
#             binned = binned.reshape((binned.shape[0],1))
            
#         feats.append(binned)
        
#     return np.stack(feats)
    
# pre_train = np.concatenate([preproc('test', test_adc_info, "FGS1", 30*12), preproc('test', test_adc_info, "AIRS-CH0", 30)], axis=2)

### fit polynoms for each sample

In [ ]:
# def phase_detector(signal):
#     phase1, phase2 = None, None
#     best_drop = 0
#     for i in range(50//2,150//2):        
#         t1 = signal[i:i+20//2].max() - signal[i:i+20//2].min()
#         if t1 > best_drop:
#             phase1 = i+(20+5)//2
#             best_drop = t1
    
#     best_drop = 0
#     for i in range(200//2,250//2):
#         t1 = signal[i:i+20//2].max() - signal[i:i+20//2].min()
#         if t1 > best_drop:
#             phase2 = i-5//2
#             best_drop = t1
    
#     return phase1, phase2

# def try_s(signal, p1, p2, deg, s):
#     out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
#     x, y = out, signal[out].tolist()
#     x = x + list(range(p1,p2))

#     y = y + (signal[p1:p2] * (1 + s[0])).tolist()
#     z = np.polyfit(x, y, deg)
#     p = np.poly1d(z)
#     q = np.abs(p(x) - y).mean()

#     if s < 1e-4:
#         return q + 1e3

#     return q
    
# def calibrate_signal(signal):
#     p1,p2 = phase_detector(signal)

#     best_deg, best_score = 1, 1e12
#     for deg in range(1, 6):
#         f = partial(try_s, signal, p1, p2, deg)
#         r = minimize(f, [0.001], method = 'Nelder-Mead')
#         s = r.x[0]

#         out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
#         x, y = out, signal[out].tolist()
#         x = x + list(range(p1,p2))
#         y = y + (signal[p1:p2] * (1 + s)).tolist()
    
#         z = np.polyfit(x, y, deg)
#         p = np.poly1d(z)
#         q = np.abs(p(x) - y).mean()
        
#         if q < best_score:
#             best_score = q
#             best_deg = deg
        
#         print(deg, q)
            
#     z = np.polyfit(x, y, best_deg)
#     p = np.poly1d(z)

#     return s, x, y, p(x)

# def calibrate_train(signal):
#     p1,p2 = phase_detector(signal)
    
#     best_deg, best_score = 1, 1e12
#     for deg in range(1, 6):
#         f = partial(try_s, signal, p1, p2, deg)
#         r = minimize(f, [0.001], method = 'Nelder-Mead')
#         s = r.x[0]

#         out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
#         x, y = out, signal[out].tolist()
#         x = x + list(range(p1,p2))
#         y = y + (signal[p1:p2] * (1 + s)).tolist()
    
#         z = np.polyfit(x, y, deg)
#         p = np.poly1d(z)
#         q = np.abs(p(x) - y).mean()
        
#         if q < best_score:
#             best_score = q
#             best_deg = deg
            
#     z = np.polyfit(x, y, best_deg)
#     p = np.poly1d(z)
    
#     return s, p(np.arange(signal.shape[0])), p1, p2


# train = pre_train.copy()
# all_s = []
# for i in range(len(test_adc_info)):
#     signal = train[i,:,1:].mean(axis=1)
#     s, p, p1, p2 = calibrate_train(pre_train[i,:,1:].mean(axis=1))
#     all_s.append(s)
        
# #copy answer 283 times because we predict mean value
# train_s = np.repeat(np.array(all_s), 283).reshape((len(all_s), 283))        
# train_sigma = np.ones_like(train_s) * 0.000176

Probably we can accurately estimate sigma from train

In [ ]:
# n = 0
# s, x, y, y_new = calibrate_signal(pre_train[n,:,1:].mean(axis=1))
# plt.scatter(x,y)
# plt.scatter(x,y_new)

I call the orange line "starline". This is probably what we would see if the planet weren't in the way.

### Making submission

In [ ]:
# ss = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv')

# preds = train_s.clip(0)
# sigmas = train_sigma
# submission = pd.DataFrame(np.concatenate([preds,sigmas], axis=1), columns=ss.columns[1:])
# submission.index = test_adc_info.index
# submission.to_csv('submission_2.csv')

## [Fork of NeurIPS Ariel 2024 - Starter 5be123](https://www.kaggle.com/code/regisvargas/fork-of-neurips-ariel-2024-starter-5be123)

### [Regis Vargas](https://www.kaggle.com/regisvargas)

References

1. [Why Did Calibration Lead to a Lower Public Score When Combining Two Kaggle Notebooks?](https://www.kaggle.com/competitions/ariel-data-challenge-2024/discussion/530472)

2. [Fork of NeurIPS Ariel 2024 - Starter 5be123](https://www.kaggle.com/code/regisvargas/fork-of-neurips-ariel-2024-starter-5be123)

3. [NeurIPS Ariel 2024 - Starter withdifferentparametr](https://www.kaggle.com/code/bingyuniu/neurips-ariel-2024-starter-withdifferentparametr)

4. [[UPDATE]Calibrating and Binning Astronomical Data](https://www.kaggle.com/code/gordonyip/update-calibrating-and-binning-astronomical-data)

5. [[UPDATE]Calibrating and Binning Astronomical Data (copy)](https://www.kaggle.com/code/aaronjday/update-calibrating-and-binning-astronomical-data)

### Initialization

This competition seems requires strong scientific background and I had lot of confusion during EDA process. Therefore, I just build a simple starter for future coding.

#### Load Library

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# import polars as pl
# import numpy as np
# import torch
# # import torch.nn as nn
# # import torch.optim as optim
# # from torch.utils.data import DataLoader, Dataset
# from tqdm import tqdm
# import pickle
# import time
# import os
# import pickle
# import seaborn as sns
# import scipy.stats
# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import Ridge
# from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# # Load Meta-Data
# PATH = "/kaggle/input/ariel-data-challenge-2024"
# train_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_adc_info.csv', 
#                              index_col='planet_id')
# train_labels = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
#                            index_col='planet_id')
# wavelengths = pd.read_csv(f'{PATH}/wavelengths.csv')
# axis_info = pd.read_parquet(os.path.join(PATH,'axis_info.parquet'))

In [ ]:
# train_adc_info['AIRS-CH0_adc_gain'].loc[785834]

### Pre-Processing
#### Load Functions

In [ ]:
# import pywt
# def wavelet_denoising(data, wavelet = 'db10', sigma=None):
#     """Denoises the signal using SURE wavelet shrinkage with the specified wavelet."""
#     # Criar uma cópia do array para evitar o erro de "read-only"
#     data = np.array(data, copy=True)
    
#     # Decompose the signal using discrete wavelet transform
#     coeffs = pywt.wavedec(data, wavelet)
    
#     # Estimate noise level if sigma is not provided
#     if sigma is None:
#         # Using the Median Absolute Deviation (MAD) estimator for noise level
#         sigma = np.median(np.abs(coeffs[-1])) / 0.6745
#     # Apply thresholding (SURE or hard/soft)
#     threshold = sigma * np.sqrt(2 * np.log(len(data)))
#     new_coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
#     # Reconstruct the signal using the modified coefficients
#     return pywt.waverec(new_coeffs, wavelet)

In [ ]:
# %%writefile utils.py
# import pandas as pd
# import polars as pl
# import numpy as np
# from tqdm import tqdm
# import pickle
# PATH = "/kaggle/input/ariel-data-challenge-2024"
# cut_inf, cut_sup = 39, 321
# def load_signal_data(planet_id, dataset, instrument, img_size):
#     file_path = f'{PATH}/{dataset}/{planet_id}/{instrument}_signal.parquet'
#     signal = pd.read_parquet(file_path)
#     if instrument == "AIRS-CH0":
#         signal = signal.values.astype(np.float64).reshape((signal.shape[0], 32, 356))
#     else:
#         signal = signal.values.astype(np.float64).reshape((signal.shape[0], 32, 32))
#     if dataset == 'train':
#         gain = train_adc_info[instrument+'_adc_gain'].loc[planet_id]
#         offset = train_adc_info[instrument+'_adc_offset'].loc[planet_id]
#         signal = ADC_convert(signal, gain, offset)
#     else:
#         gain = test_adc_info[instrument+'_adc_gain'].loc[planet_id]
#         offset = test_adc_info[instrument+'_adc_offset'].loc[planet_id]
#         signal = ADC_convert(signal, gain, offset)
#     if instrument == "AIRS-CH0":
#         dt_airs = axis_info['AIRS-CH0-integration_time'].dropna().values
#         dt_airs[1::2] += 0.1
#         signal = signal[:, :, cut_inf:cut_sup]
#     signal = signal.reshape(signal.shape[0], signal.shape[1] * signal.shape[2])
#     mean_signal = signal.mean(axis=1)
#     mean_signal = mean_signal / np.linalg.norm(mean_signal)
#     net_signal = mean_signal[1::2] - mean_signal[0::2]
#     #return wavelet_denoising(net_signal)
#     return net_signal
# def ADC_convert(signal, gain, offset):
#     signal = signal.astype(np.float64)
#     signal /= gain
#     signal += offset
#     return signal
# def read_and_preprocess(dataset, planet_ids, instrument = "AIRS-CH0"):
#     """Read the files for all planet_ids and extract the time series.
#     Parameters
#     dataset: 'train' or 'test'
#     planet_ids: list of planet ids
#     instrument: the instrument of observation, 'AIRS-CH0' or 'FGS1', default to 'AIRS-CH0'
#     Returns
#     dataframe with one row per planet_id and 67500 values per row for FGS1 and 5624 for AIRS-CH0
#     """
#     img_size = 1024 if instrument == "FGS1" else 32*356
#     column_num = 67500 if instrument == 'FGS1' else 5625
#     raw_train = np.full((len(planet_ids), column_num), np.nan, dtype=np.float32)
#     for i, planet_id in tqdm(list(enumerate(planet_ids))):
#         raw_train[i] = load_signal_data(planet_id, dataset, instrument, img_size)
#     return raw_train
# def feature_engineering(f_raw, a_raw, adc_info, window_size=50, step_size=15):
#     """Create a dataframe with combined features from the raw data, including sliding window and time-series statistics.
    
#     Parameters:
#     f_raw: ndarray of shape (n_planets, 67500)
#     a_raw: ndarray of shape (n_planets, 5625)
#     window_size: int, size of the sliding window for time-series statistics
#     step_size: int, step size for the sliding window
    
#     Return value:
#     df: DataFrame of shape (n_planets, several features)
#     """
#     f_obscured = f_raw[:, 23500:44000].mean(axis=1)
#     f_unobscured = (f_raw[:, :20500].mean(axis=1) + f_raw[:, 47000:].mean(axis=1)) / 2
#     f_relative_reduction = (f_unobscured - f_obscured) / f_unobscured
#     f_std_dev = f_raw.std(axis=1)
#     f_signal_to_noise = f_unobscured / f_std_dev
#     a_obscured = a_raw[:, 1958:3666].mean(axis=1)
#     a_unobscured = (a_raw[:, :1708].mean(axis=1) + a_raw[:, 3916:].mean(axis=1)) / 2
#     a_relative_reduction = (a_unobscured - a_obscured) / a_unobscured
#     a_std_dev = a_raw.std(axis=1)
#     a_signal_to_noise = a_unobscured / a_std_dev
#     f_variance = f_raw.var(axis=1)
#     a_variance = a_raw.var(axis=1)
    
#     f_skewness = pd.DataFrame(f_raw).skew(axis=1).values
#     a_skewness = pd.DataFrame(a_raw).skew(axis=1).values
#     f_kurtosis = pd.DataFrame(f_raw).kurtosis(axis=1).values
#     a_kurtosis = pd.DataFrame(a_raw).kurtosis(axis=1).values
    
#     f_half_obscured1 = f_raw[:, 20500:23500].mean(axis=1)
#     f_half_obscured2 = f_raw[:, 44000:47000].mean(axis=1)
#     f_half_reduction1 = (f_unobscured - f_half_obscured1) / f_unobscured
#     f_half_reduction2 = (f_unobscured - f_half_obscured2) / f_unobscured
#     a_half_obscured1 = a_raw[:, 1708:1958].mean(axis=1)
#     a_half_obscured2 = a_raw[:, 3666:3916].mean(axis=1)
#     a_half_reduction1 = (a_unobscured - a_half_obscured1) / a_unobscured
#     a_half_reduction2 = (a_unobscured - a_half_obscured2) / a_unobscured
#     # Sliding window features
#     def sliding_window_features(data, window_size, step_size):
#         features = []
#         max_index = data.shape[1]
#         for start in range(0, max_index - window_size + 1, step_size):
#             end = start + window_size
#             window = data[:, start:end]
#             features.append([
#                 np.mean(window, axis=1),
#                 np.std(window, axis=1),
#                 np.min(window, axis=1),
#                 np.max(window, axis=1)
#             ])
#         if features:
#             return np.vstack(features).T  # Stack vertically and transpose to get the correct shape
#         else:
#             return np.empty((data.shape[0], 0))  # Return empty array with correct shape
    
#     f_sliding_features = sliding_window_features(f_raw, window_size, step_size)
#     a_sliding_features = sliding_window_features(a_raw, window_size, step_size)
#     print(f'f_sliding_features.shape: {f_sliding_features.shape}')
#     print(f'a_sliding_features.shape: {a_sliding_features.shape}')
#     df = pd.DataFrame({
#         'f_relative_reduction': f_relative_reduction,
#         'f_signal_to_noise': f_signal_to_noise,
#         'f_variance': f_variance,
#         'f_skewness': f_skewness,
#         'f_kurtosis': f_kurtosis,
#         'a_relative_reduction': a_relative_reduction,
#         'a_signal_to_noise': a_signal_to_noise,
#         'a_variance': a_variance,
#         'a_skewness': a_skewness,
#         'a_kurtosis': a_kurtosis,
#         'f_half_reduction1': f_half_reduction1,
#         'f_half_reduction2': f_half_reduction2,
#         'a_half_reduction1': a_half_reduction1,
#         'a_half_reduction2': a_half_reduction2
#     })
#     if f_sliding_features.size > 0:
#         f_sliding_df = pd.DataFrame(f_sliding_features, columns=[f'f_slide_{i}' for i in range(f_sliding_features.shape[1])])
#         df = pd.concat([df, f_sliding_df], axis=1)
#     if a_sliding_features.size > 0:
#         a_sliding_df = pd.DataFrame(a_sliding_features, columns=[f'a_slide_{i}' for i in range(a_sliding_features.shape[1])])
#         df = pd.concat([df, a_sliding_df], axis=1)
    
#     df = pd.concat([df, adc_info.reset_index().iloc[:, 1:6]], axis=1)
    
#     return df

In [ ]:
# %%writefile -a utils.py

# def postprocessing(pred_array, index, sigma_pred):
#     """Create a submission dataframe from its components
    
#     Parameters:
#     pred_array: ndarray of shape (n_samples, 283)
#     index: pandas.Index of length n_samples with name 'planet_id'
#     sigma_pred: float
    
#     Return value:
#     df: DataFrame of shape (n_samples, 566) with planet_id as index
#     """
#     return pd.concat([pd.DataFrame(pred_array.clip(0, None), index=index, columns=wavelengths.columns),
#                       pd.DataFrame(sigma_pred, index=index, columns=[f"sigma_{i}" for i in range(1, 284)])],
#                      axis=1)

# class ParticipantVisibleError(Exception):
#     pass

# def competition_score(
#         solution: pd.DataFrame,
#         submission: pd.DataFrame,
#         naive_mean: float,
#         naive_sigma: float,
#         sigma_true: float,
#         row_id_column_name='planet_id',
#     ) -> float:
#     '''
#     This is a Gaussian Log Likelihood based metric. For a submission, which contains the predicted mean (x_hat) and variance (x_hat_std),
#     we calculate the Gaussian Log-likelihood (GLL) value to the provided ground truth (x). We treat each pair of x_hat,
#     x_hat_std as a 1D gaussian, meaning there will be 283 1D gaussian distributions, hence 283 values for each test spectrum,
#     the GLL value for one spectrum is the sum of all of them.

#     Inputs:
#         - solution: Ground Truth spectra (from test set)
#             - shape: (nsamples, n_wavelengths)
#         - submission: Predicted spectra and errors (from participants)
#             - shape: (nsamples, n_wavelengths*2)
#         naive_mean: (float) mean from the train set.
#         naive_sigma: (float) standard deviation from the train set.
#         sigma_true: (float) essentially sets the scale of the outputs.
#     '''

#     del solution[row_id_column_name]
#     del submission[row_id_column_name]

#     if submission.min().min() < 0:
#         raise ParticipantVisibleError('Negative values in the submission')
#     for col in submission.columns:
#         if not pd.api.types.is_numeric_dtype(submission[col]):
#             raise ParticipantVisibleError(f'Submission column {col} must be a number')

#     n_wavelengths = len(solution.columns)
#     if len(submission.columns) != n_wavelengths*2:
#         raise ParticipantVisibleError('Wrong number of columns in the submission')

#     y_pred = submission.iloc[:, :n_wavelengths].values
#     # Set a non-zero minimum sigma pred to prevent division by zero errors.
#     sigma_pred = np.clip(submission.iloc[:, n_wavelengths:].values, a_min=10**-15, a_max=None)
#     y_true = solution.values

#     GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred))
#     GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)))
#     GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=naive_mean * np.ones_like(y_true), scale=naive_sigma * np.ones_like(y_true)))

#     submit_score = (GLL_pred - GLL_mean)/(GLL_true - GLL_mean)
#     return float(np.clip(submit_score, 0.0, 1.0))

In [ ]:
# exec(open('utils.py', 'r').read())

### Load Data

In [ ]:
# %%time
# if os.path.exists("/kaggle/input/adc24-intro-training/f_raw_train.pickle"):
#     f_raw_train = np.load('/kaggle/input/adc24-intro-training/f_raw_train.pickle', allow_pickle=True)
# else:
#     f_raw_train = read_and_preprocess('train', train_labels.index, 'FGS1')
#     with open('f_raw_train.pickle', 'wb') as f:
#         pickle.dump(f_raw_train, f)

In [ ]:
# %%time
# if os.path.exists("/kaggle/input/adc24-intro-training/a_raw_train.pickle"):
#     a_raw_train = np.load('/kaggle/input/adc24-intro-training/a_raw_train.pickle', allow_pickle=True)
# else:
#     a_raw_train = read_and_preprocess('train', train_labels.index)
#     with open('a_raw_train.pickle', 'wb') as f:
#         pickle.dump(a_raw_train, f)

### Feature Engineering

In [ ]:
# %%time
# train = feature_engineering(f_raw_train, a_raw_train, train_adc_info)

In [ ]:
# train.head()

In [ ]:
# train = train.iloc[:,:-1]

### Data Plot

In [ ]:
# plt.figure(figsize=(6, 2))
# plt.plot(f_raw_train.mean(axis=0))
# for time_step in [20500, 23500, 44000, 47000]:
#     plt.axvline(time_step, color='gray')
# plt.xlabel('time step')
# plt.title('FGS1: Overall mean')
# plt.show()

# plt.figure(figsize=(6, 2))
# plt.plot(a_raw_train.mean(axis=0))
# for time_step in [20500, 23500, 44000, 47000]:
#     plt.axvline(time_step * 11250 // 135000, color='gray')
# plt.xlabel('time step')
# plt.title('AIRS-CH0: Overall mean')
# plt.show()

In [ ]:
# color_array = np.array(plt.rcParams['axes.prop_cycle'].by_key()['color'])
# plt.scatter(train.a_relative_reduction, train_labels.wl_1, s=15, alpha=0.5,
#             c=color_array[train_adc_info.star])
# plt.xlabel('relative signal reduction when planet is in front')
# plt.ylabel('target')
# plt.title('Correlation between relative signal reduction and target')
# # plt.gca().set_aspect('equal')
# points = [plt.Line2D([0], [0], label=f'star {i}', marker='o', markersize=3,
#          markeredgecolor=color_array[i], markerfacecolor=color_array[i], linestyle='') for i in range(2)]

# plt.legend(handles=points)
# plt.show()

### Model
#### Rigde Model

In [ ]:
# model = Ridge(alpha=1e-12)

# oof_pred = cross_val_predict(model, train, train_labels)

# print(f"# R2 score: {r2_score(train_labels, oof_pred):.4f}")
# sigma_pred = mean_squared_error(train_labels, oof_pred, squared=False)
# print(f"# Root mean squared error: {sigma_pred:.7f}")

In [ ]:
# oof_df = postprocessing(oof_pred, train_adc_info.index, sigma_pred)
# display(oof_df)

# gll_score = competition_score(train_labels.copy().reset_index(),
#                               oof_df.copy().reset_index(),
#                               naive_mean=train_labels.values.mean(),
#                               naive_sigma=train_labels.values.std(),
#                               sigma_true=0.000003)
# print(f"# Estimated competition score: {gll_score:.4f}")

In [ ]:
# model.fit(train, train_labels)
# with open('model.pickle', 'wb') as f:
#     pickle.dump(model, f)
# with open('sigma_pred.pickle', 'wb') as f:
#     pickle.dump(sigma_pred, f)

### Inference

In [ ]:
# # Load the data
# test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
#                            index_col='planet_id')
# sample_submission = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv',
#                                 index_col='planet_id')
# f_raw_test = read_and_preprocess('test', sample_submission.index, 'FGS1')
# a_raw_test = read_and_preprocess('test', sample_submission.index)
# test = feature_engineering(f_raw_test, a_raw_test, test_adc_info)
# test = test.iloc[: , :-1]
# # Load the model
# with open('model.pickle', 'rb') as f:
#     model = pickle.load(f)
# with open('sigma_pred.pickle', 'rb') as f:
#     sigma_pred = pickle.load(f)

# # Predict
# test_pred = model.predict(test)

# # Package into submission file
# sub_df = sub_df = postprocessing(test_pred,
#                         test_adc_info.index,
#                         sigma_pred=np.tile(np.where(test_adc_info[['star']] <= 1, 0.0001555, 0.00085), (1, 283)))
# display(sub_df)
# sub_df.to_csv('submission_3.csv')

## [AD24[TRAIN-INF]Ridge_AddFE[LB.441]](https://www.kaggle.com/code/hideyukizushi/ad24-train-inf-ridge-addfe-lb-441)

### [yukiZ](https://www.kaggle.com/hideyukizushi)

### ℹ️ **Info**
* **forked original great work kernels**

    * https://www.kaggle.com/code/ambrosm/adc24-intro-inference
    * https://www.kaggle.com/code/hugowjd/neurips-ariel-2024-starter
    * https://www.kaggle.com/code/xiaocao123/neurips-ariel-2024-starter?scriptVersionId=193156800
    * https://www.kaggle.com/code/bingyuniu/neurips-ariel-2024-starter-withdifferentparametr

* **2024/08/26 My Additional**
    * percentile FE add.
```
np.percentile(window, 5, axis=1),
np.percentile(window, 10, axis=1),
np.percentile(window, 15, axis=1),
np.percentile(window, 20, axis=1),
np.percentile(window, 25, axis=1),
np.percentile(window, 30, axis=1),
np.percentile(window, 35, axis=1),
np.percentile(window, 40, axis=1),
np.percentile(window, 60, axis=1),
np.percentile(window, 65, axis=1),
np.percentile(window, 70, axis=1),
np.percentile(window, 75, axis=1),
np.percentile(window, 80, axis=1),
np.percentile(window, 85, axis=1),
np.percentile(window, 90, axis=1),
np.percentile(window, 95, axis=1),
np.median(window, axis=1),
np.var(window, axis=1),
```

* **2024/08/29 My Additional**
    * add FE
```
f_sliding_features2 = sliding_window_features(f_raw, 100, 10)
a_sliding_features2 = sliding_window_features(a_raw, 100, 10)
```

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# import polars as pl
# import numpy as np
# import torch
# # import torch.nn as nn
# # import torch.optim as optim
# # from torch.utils.data import DataLoader, Dataset
# from tqdm import tqdm
# import pickle
# import time
# import os
# import pickle
# import seaborn as sns
# import scipy.stats
# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import Ridge
# from sklearn.metrics import r2_score, mean_squared_error
# from scipy.stats import kurtosis
# from scipy.stats import skew

In [ ]:
# # Load Meta-Data
# PATH = "/kaggle/input/ariel-data-challenge-2024"
# train_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_adc_info.csv', 
#                              index_col='planet_id')
# train_labels = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
#                            index_col='planet_id')
# wavelengths = pd.read_csv(f'{PATH}/wavelengths.csv')

### Pre-Processing
#### Load Functions

In [ ]:
# %%writefile utils.py
# import pandas as pd
# import polars as pl
# import numpy as np
# from tqdm import tqdm
# import pickle
# PATH = "/kaggle/input/ariel-data-challenge-2024"
# def load_signal_data(planet_id, dataset, instrument, img_size):
#     file_path = f'{PATH}/{dataset}/{planet_id}/{instrument}_signal.parquet'
#     signal = pl.read_parquet(file_path)
#     mean_signal = signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / img_size # mean over the 32*32 pixels
#     net_signal = mean_signal[1::2] - mean_signal[0::2]
#     return net_signal

# def read_and_preprocess(dataset, planet_ids, instrument = "AIRS-CH0"):
#     """Read the files for all planet_ids and extract the time series.
#     Parameters
#     dataset: 'train' or 'test'
#     planet_ids: list of planet ids
#     instrument: the instrument of observation, 'AIRS-CH0' or 'FGS1', default to 'AIRS-CH0'
#     Returns
#     dataframe with one row per planet_id and 67500 values per row for FGS1 and 5624 for AIRS-CH0
#     """
#     img_size = 1024 if instrument == "FGS1" else 32*356
#     column_num = 67500 if instrument == 'FGS1' else 5625
#     raw_train = np.full((len(planet_ids), column_num), np.nan, dtype=np.float32)
#     for i, planet_id in tqdm(list(enumerate(planet_ids))):
#         raw_train[i] = load_signal_data(planet_id, dataset, instrument, img_size)
#     return raw_train

# def feature_engineering(f_raw, a_raw, adc_info, window_size=60, step_size=15):
#     """Create a dataframe with combined features from the raw data, including sliding window and time-series statistics.
    
#     Parameters:
#     f_raw: ndarray of shape (n_planets, 67500)
#     a_raw: ndarray of shape (n_planets, 5625)
#     window_size: int, size of the sliding window for time-series statistics
#     step_size: int, step size for the sliding window
    
#     Return value:
#     df: DataFrame of shape (n_planets, several features)
#     """
#     f_obscured = f_raw[:, 23500:44000].mean(axis=1)
#     f_unobscured = (f_raw[:, :20500].mean(axis=1) + f_raw[:, 47000:].mean(axis=1)) / 2
#     f_relative_reduction = (f_unobscured - f_obscured) / f_unobscured
#     f_std_dev = f_raw.std(axis=1)
#     f_signal_to_noise = f_unobscured / f_std_dev

#     a_obscured = a_raw[:, 1958:3666].mean(axis=1)
#     a_unobscured = (a_raw[:, :1708].mean(axis=1) + a_raw[:, 3916:].mean(axis=1)) / 2
#     a_relative_reduction = (a_unobscured - a_obscured) / a_unobscured
#     a_std_dev = a_raw.std(axis=1)
#     a_signal_to_noise = a_unobscured / a_std_dev

#     f_variance = f_raw.var(axis=1)
#     a_variance = a_raw.var(axis=1)
    
#     f_skewness = pd.DataFrame(f_raw).skew(axis=1).values
#     a_skewness = pd.DataFrame(a_raw).skew(axis=1).values

#     f_kurtosis = pd.DataFrame(f_raw).kurtosis(axis=1).values
#     a_kurtosis = pd.DataFrame(a_raw).kurtosis(axis=1).values
    
#     f_half_obscured1 = f_raw[:, 20500:23500].mean(axis=1)
#     f_half_obscured2 = f_raw[:, 44000:47000].mean(axis=1)
#     f_half_reduction1 = (f_unobscured - f_half_obscured1) / f_unobscured
#     f_half_reduction2 = (f_unobscured - f_half_obscured2) / f_unobscured

#     a_half_obscured1 = a_raw[:, 1708:1958].mean(axis=1)
#     a_half_obscured2 = a_raw[:, 3666:3916].mean(axis=1)
#     a_half_reduction1 = (a_unobscured - a_half_obscured1) / a_unobscured
#     a_half_reduction2 = (a_unobscured - a_half_obscured2) / a_unobscured

#     # Sliding window features
#     def sliding_window_features(data, window_size, step_size):
#         features = []
#         max_index = data.shape[1]
#         for start in range(0, max_index - window_size + 1, step_size):
#             end = start + window_size
#             window = data[:, start:end]
#             features.append([
#                 np.mean(window, axis=1),
#                 np.std(window, axis=1),
#                 np.min(window, axis=1),
#                 np.max(window, axis=1),
#                 np.percentile(window, 5, axis=1),
#                 np.percentile(window, 10, axis=1),
#                 np.percentile(window, 15, axis=1),
#                 np.percentile(window, 20, axis=1),
#                 np.percentile(window, 25, axis=1),
#                 np.percentile(window, 30, axis=1),
#                 np.percentile(window, 35, axis=1),
#                 np.percentile(window, 40, axis=1),
#                 np.percentile(window, 60, axis=1),
#                 np.percentile(window, 65, axis=1),
#                 np.percentile(window, 70, axis=1),
#                 np.percentile(window, 75, axis=1),
#                 np.percentile(window, 80, axis=1),
#                 np.percentile(window, 85, axis=1),
#                 np.percentile(window, 90, axis=1),
#                 np.percentile(window, 95, axis=1),
#                 np.median(window, axis=1),
#                 np.var(window, axis=1),
# #                 kurtosis(window, axis=1),
# #                 skew(window, axis=1),
#             ])
#         if features:
#             return np.vstack(features).T  # Stack vertically and transpose to get the correct shape
#         else:
#             return np.empty((data.shape[0], 0))  # Return empty array with correct shape
    
#     f_sliding_features = sliding_window_features(f_raw, window_size, step_size)
#     a_sliding_features = sliding_window_features(a_raw, window_size, step_size)
    
#     f_sliding_features2 = sliding_window_features(f_raw, 100, 10)
#     a_sliding_features2 = sliding_window_features(a_raw, 100, 10)


#     print(f'f_sliding_features.shape: {f_sliding_features.shape}')
#     print(f'a_sliding_features.shape: {a_sliding_features.shape}')


#     df = pd.DataFrame({
#         'f_relative_reduction': f_relative_reduction,
#         'f_signal_to_noise': f_signal_to_noise,
#         'f_variance': f_variance,
#         'f_skewness': f_skewness,
#         'f_kurtosis': f_kurtosis,
#         'a_relative_reduction': a_relative_reduction,
#         'a_signal_to_noise': a_signal_to_noise,
#         'a_variance': a_variance,
#         'a_skewness': a_skewness,
#         'a_kurtosis': a_kurtosis,
#         'f_half_reduction1': f_half_reduction1,
#         'f_half_reduction2': f_half_reduction2,
#         'a_half_reduction1': a_half_reduction1,
#         'a_half_reduction2': a_half_reduction2
#     })


#     if f_sliding_features.size > 0:
#         f_sliding_df = pd.DataFrame(f_sliding_features, columns=[f'f_slide_{i}' for i in range(f_sliding_features.shape[1])])
#         df = pd.concat([df, f_sliding_df], axis=1)
#     if a_sliding_features.size > 0:
#         a_sliding_df = pd.DataFrame(a_sliding_features, columns=[f'a_slide_{i}' for i in range(a_sliding_features.shape[1])])
#         df = pd.concat([df, a_sliding_df], axis=1)
    
#     if f_sliding_features2.size > 0:
#         f_sliding_df = pd.DataFrame(f_sliding_features2, columns=[f'f_slide2_{i}' for i in range(f_sliding_features2.shape[1])])
#         df = pd.concat([df, f_sliding_df], axis=1)
#     if a_sliding_features2.size > 0:
#         a_sliding_df = pd.DataFrame(a_sliding_features2, columns=[f'a_slide2_{i}' for i in range(a_sliding_features2.shape[1])])
#         df = pd.concat([df, a_sliding_df], axis=1)
    
    
#     df = pd.concat([df, adc_info.reset_index().iloc[:, 1:6]], axis=1)
    
#     return df

In [ ]:
# %%writefile -a utils.py

# def postprocessing(pred_array, index, sigma_pred):
#     """Create a submission dataframe from its components
    
#     Parameters:
#     pred_array: ndarray of shape (n_samples, 283)
#     index: pandas.Index of length n_samples with name 'planet_id'
#     sigma_pred: float
    
#     Return value:
#     df: DataFrame of shape (n_samples, 566) with planet_id as index
#     """
#     return pd.concat([pd.DataFrame(pred_array.clip(0, None), index=index, columns=wavelengths.columns),
#                       pd.DataFrame(sigma_pred, index=index, columns=[f"sigma_{i}" for i in range(1, 284)])],
#                      axis=1)

# class ParticipantVisibleError(Exception):
#     pass

# def competition_score(
#         solution: pd.DataFrame,
#         submission: pd.DataFrame,
#         naive_mean: float,
#         naive_sigma: float,
#         sigma_true: float,
#         row_id_column_name='planet_id',
#     ) -> float:
#     '''
#     This is a Gaussian Log Likelihood based metric. For a submission, which contains the predicted mean (x_hat) and variance (x_hat_std),
#     we calculate the Gaussian Log-likelihood (GLL) value to the provided ground truth (x). We treat each pair of x_hat,
#     x_hat_std as a 1D gaussian, meaning there will be 283 1D gaussian distributions, hence 283 values for each test spectrum,
#     the GLL value for one spectrum is the sum of all of them.

#     Inputs:
#         - solution: Ground Truth spectra (from test set)
#             - shape: (nsamples, n_wavelengths)
#         - submission: Predicted spectra and errors (from participants)
#             - shape: (nsamples, n_wavelengths*2)
#         naive_mean: (float) mean from the train set.
#         naive_sigma: (float) standard deviation from the train set.
#         sigma_true: (float) essentially sets the scale of the outputs.
#     '''

#     del solution[row_id_column_name]
#     del submission[row_id_column_name]

#     if submission.min().min() < 0:
#         raise ParticipantVisibleError('Negative values in the submission')
#     for col in submission.columns:
#         if not pd.api.types.is_numeric_dtype(submission[col]):
#             raise ParticipantVisibleError(f'Submission column {col} must be a number')

#     n_wavelengths = len(solution.columns)
#     if len(submission.columns) != n_wavelengths*2:
#         raise ParticipantVisibleError('Wrong number of columns in the submission')

#     y_pred = submission.iloc[:, :n_wavelengths].values
#     # Set a non-zero minimum sigma pred to prevent division by zero errors.
#     sigma_pred = np.clip(submission.iloc[:, n_wavelengths:].values, a_min=10**-15, a_max=None)
#     y_true = solution.values

#     GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred))
#     GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)))
#     GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=naive_mean * np.ones_like(y_true), scale=naive_sigma * np.ones_like(y_true)))

#     submit_score = (GLL_pred - GLL_mean)/(GLL_true - GLL_mean)
#     return float(np.clip(submit_score, 0.0, 1.0))

In [ ]:
# exec(open('utils.py', 'r').read())

### Load Data

In [ ]:
# %%time
# if os.path.exists("/kaggle/input/adc24-intro-training/f_raw_train.pickle"):
#     f_raw_train = np.load('/kaggle/input/adc24-intro-training/f_raw_train.pickle', allow_pickle=True)
# else:
#     f_raw_train = read_and_preprocess('train', train_labels.index, 'FGS1')
#     with open('f_raw_train.pickle', 'wb') as f:
#         pickle.dump(f_raw_train, f)

In [ ]:
# %%time
# if os.path.exists("/kaggle/input/adc24-intro-training/a_raw_train.pickle"):
#     a_raw_train = np.load('/kaggle/input/adc24-intro-training/a_raw_train.pickle', allow_pickle=True)
# else:
#     a_raw_train = read_and_preprocess('train', train_labels.index)
#     with open('a_raw_train.pickle', 'wb') as f:
#         pickle.dump(a_raw_train, f)

### Feature Engineering

In [ ]:
# %%time
# train = feature_engineering(f_raw_train, a_raw_train, train_adc_info)

In [ ]:
# train.head()

In [ ]:
# train = train.iloc[:,:-1]

### Data Plot

In [ ]:
# plt.figure(figsize=(6, 2))
# plt.plot(f_raw_train.mean(axis=0))
# for time_step in [20500, 23500, 44000, 47000]:
#     plt.axvline(time_step, color='gray')
# plt.xlabel('time step')
# plt.title('FGS1: Overall mean')
# plt.show()

# plt.figure(figsize=(6, 2))
# plt.plot(a_raw_train.mean(axis=0))
# for time_step in [20500, 23500, 44000, 47000]:
#     plt.axvline(time_step * 11250 // 135000, color='gray')
# plt.xlabel('time step')
# plt.title('AIRS-CH0: Overall mean')
# plt.show()

In [ ]:
# color_array = np.array(plt.rcParams['axes.prop_cycle'].by_key()['color'])
# plt.scatter(train.a_relative_reduction, train_labels.wl_1, s=15, alpha=0.5,
#             c=color_array[train_adc_info.star])
# plt.xlabel('relative signal reduction when planet is in front')
# plt.ylabel('target')
# plt.title('Correlation between relative signal reduction and target')
# # plt.gca().set_aspect('equal')
# points = [plt.Line2D([0], [0], label=f'star {i}', marker='o', markersize=3,
#          markeredgecolor=color_array[i], markerfacecolor=color_array[i], linestyle='') for i in range(2)]

# plt.legend(handles=points)
# plt.show()

### Model
#### Rigde Model

In [ ]:
# model = Ridge(alpha=1e-12)

# oof_pred = cross_val_predict(model, train, train_labels)

# print(f"# R2 score: {r2_score(train_labels, oof_pred):.4f}")
# sigma_pred = mean_squared_error(train_labels, oof_pred, squared=False)
# print(f"# Root mean squared error: {sigma_pred:.7f}")

In [ ]:
# oof_df = postprocessing(oof_pred, train_adc_info.index, sigma_pred)
# display(oof_df)

# gll_score = competition_score(train_labels.copy().reset_index(),
#                               oof_df.copy().reset_index(),
#                               naive_mean=train_labels.values.mean(),
#                               naive_sigma=train_labels.values.std(),
#                               sigma_true=0.000003)
# print(f"# Estimated competition score: {gll_score:.4f}")

In [ ]:
# model.fit(train, train_labels)
# with open('model.pickle', 'wb') as f:
#     pickle.dump(model, f)
# with open('sigma_pred.pickle', 'wb') as f:
#     pickle.dump(sigma_pred, f)

### Inference

In [ ]:
# # Load the data
# test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
#                            index_col='planet_id')
# sample_submission = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv',
#                                 index_col='planet_id')
# f_raw_test = read_and_preprocess('test', sample_submission.index, 'FGS1')
# a_raw_test = read_and_preprocess('test', sample_submission.index)
# test = feature_engineering(f_raw_test, a_raw_test, test_adc_info)
# test = test.iloc[: , :-1]
# # Load the model
# with open('model.pickle', 'rb') as f:
#     model = pickle.load(f)
# with open('sigma_pred.pickle', 'rb') as f:
#     sigma_pred = pickle.load(f)

# # Predict
# test_pred = model.predict(test)

# # Package into submission file
# sub_df = sub_df = postprocessing(test_pred,
#                         test_adc_info.index,
#                         sigma_pred=np.tile(np.where(test_adc_info[['star']] <= 1, 0.0001555, 0.00085), (1, 283)))
# display(sub_df)
# sub_df.to_csv('submission_4.csv')

## [NeurIPS Ariel 2024 - Starter withdifferentparametr](https://www.kaggle.com/code/hideyukizushi/ad24-train-inf-ridge-addfe-lb-441)

### [yu🐂](https://www.kaggle.com/bingyuniu)

### Initialization

This competition seems requires strong scientific background and I had lot of confusion during EDA process. Therefore, I just build a simple starter for future coding.

#### Load Library

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# import polars as pl
# import numpy as np
# import torch
# # import torch.nn as nn
# # import torch.optim as optim
# # from torch.utils.data import DataLoader, Dataset
# from tqdm import tqdm
# import pickle
# import time
# import os
# import pickle
# import seaborn as sns
# import scipy.stats
# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import Ridge
# from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# # Load Meta-Data
# PATH = "/kaggle/input/ariel-data-challenge-2024"
# train_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_adc_info.csv', 
#                              index_col='planet_id')
# train_labels = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
#                            index_col='planet_id')
# wavelengths = pd.read_csv(f'{PATH}/wavelengths.csv')

### Pre-Processing
#### Load Functions

In [ ]:
# %%writefile utils.py
# import pandas as pd
# import polars as pl
# import numpy as np
# from tqdm import tqdm
# import pickle
# PATH = "/kaggle/input/ariel-data-challenge-2024"
# def load_signal_data(planet_id, dataset, instrument, img_size):
#     file_path = f'{PATH}/{dataset}/{planet_id}/{instrument}_signal.parquet'
#     signal = pl.read_parquet(file_path)
#     mean_signal = signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / img_size # mean over the 32*32 pixels
#     net_signal = mean_signal[1::2] - mean_signal[0::2]
#     return net_signal

# def read_and_preprocess(dataset, planet_ids, instrument = "AIRS-CH0"):
#     """Read the files for all planet_ids and extract the time series.
#     Parameters
#     dataset: 'train' or 'test'
#     planet_ids: list of planet ids
#     instrument: the instrument of observation, 'AIRS-CH0' or 'FGS1', default to 'AIRS-CH0'
#     Returns
#     dataframe with one row per planet_id and 67500 values per row for FGS1 and 5624 for AIRS-CH0
#     """
#     img_size = 1024 if instrument == "FGS1" else 32*356
#     column_num = 67500 if instrument == 'FGS1' else 5625
#     raw_train = np.full((len(planet_ids), column_num), np.nan, dtype=np.float32)
#     for i, planet_id in tqdm(list(enumerate(planet_ids))):
#         raw_train[i] = load_signal_data(planet_id, dataset, instrument, img_size)
#     return raw_train

# def feature_engineering(f_raw, a_raw, adc_info, window_size=50, step_size=15):
#     """Create a dataframe with combined features from the raw data, including sliding window and time-series statistics.
    
#     Parameters:
#     f_raw: ndarray of shape (n_planets, 67500)
#     a_raw: ndarray of shape (n_planets, 5625)
#     window_size: int, size of the sliding window for time-series statistics
#     step_size: int, step size for the sliding window
    
#     Return value:
#     df: DataFrame of shape (n_planets, several features)
#     """
#     f_obscured = f_raw[:, 23500:44000].mean(axis=1)
#     f_unobscured = (f_raw[:, :20500].mean(axis=1) + f_raw[:, 47000:].mean(axis=1)) / 2
#     f_relative_reduction = (f_unobscured - f_obscured) / f_unobscured
#     f_std_dev = f_raw.std(axis=1)
#     f_signal_to_noise = f_unobscured / f_std_dev

#     a_obscured = a_raw[:, 1958:3666].mean(axis=1)
#     a_unobscured = (a_raw[:, :1708].mean(axis=1) + a_raw[:, 3916:].mean(axis=1)) / 2
#     a_relative_reduction = (a_unobscured - a_obscured) / a_unobscured
#     a_std_dev = a_raw.std(axis=1)
#     a_signal_to_noise = a_unobscured / a_std_dev

#     f_variance = f_raw.var(axis=1)
#     a_variance = a_raw.var(axis=1)
    
#     f_skewness = pd.DataFrame(f_raw).skew(axis=1).values
#     a_skewness = pd.DataFrame(a_raw).skew(axis=1).values

#     f_kurtosis = pd.DataFrame(f_raw).kurtosis(axis=1).values
#     a_kurtosis = pd.DataFrame(a_raw).kurtosis(axis=1).values
    
#     f_half_obscured1 = f_raw[:, 20500:23500].mean(axis=1)
#     f_half_obscured2 = f_raw[:, 44000:47000].mean(axis=1)
#     f_half_reduction1 = (f_unobscured - f_half_obscured1) / f_unobscured
#     f_half_reduction2 = (f_unobscured - f_half_obscured2) / f_unobscured

#     a_half_obscured1 = a_raw[:, 1708:1958].mean(axis=1)
#     a_half_obscured2 = a_raw[:, 3666:3916].mean(axis=1)
#     a_half_reduction1 = (a_unobscured - a_half_obscured1) / a_unobscured
#     a_half_reduction2 = (a_unobscured - a_half_obscured2) / a_unobscured

#     # Sliding window features
#     def sliding_window_features(data, window_size, step_size):
#         features = []
#         max_index = data.shape[1]
#         for start in range(0, max_index - window_size + 1, step_size):
#             end = start + window_size
#             window = data[:, start:end]
#             features.append([
#                 np.mean(window, axis=1),
#                 np.std(window, axis=1),
#                 np.min(window, axis=1),
#                 np.max(window, axis=1)
#             ])
#         if features:
#             return np.vstack(features).T  # Stack vertically and transpose to get the correct shape
#         else:
#             return np.empty((data.shape[0], 0))  # Return empty array with correct shape
    
#     f_sliding_features = sliding_window_features(f_raw, window_size, step_size)
#     a_sliding_features = sliding_window_features(a_raw, window_size, step_size)


#     print(f'f_sliding_features.shape: {f_sliding_features.shape}')
#     print(f'a_sliding_features.shape: {a_sliding_features.shape}')


#     df = pd.DataFrame({
#         'f_relative_reduction': f_relative_reduction,
#         'f_signal_to_noise': f_signal_to_noise,
#         'f_variance': f_variance,
#         'f_skewness': f_skewness,
#         'f_kurtosis': f_kurtosis,
#         'a_relative_reduction': a_relative_reduction,
#         'a_signal_to_noise': a_signal_to_noise,
#         'a_variance': a_variance,
#         'a_skewness': a_skewness,
#         'a_kurtosis': a_kurtosis,
#         'f_half_reduction1': f_half_reduction1,
#         'f_half_reduction2': f_half_reduction2,
#         'a_half_reduction1': a_half_reduction1,
#         'a_half_reduction2': a_half_reduction2
#     })


#     if f_sliding_features.size > 0:
#         f_sliding_df = pd.DataFrame(f_sliding_features, columns=[f'f_slide_{i}' for i in range(f_sliding_features.shape[1])])
#         df = pd.concat([df, f_sliding_df], axis=1)

#     if a_sliding_features.size > 0:
#         a_sliding_df = pd.DataFrame(a_sliding_features, columns=[f'a_slide_{i}' for i in range(a_sliding_features.shape[1])])
#         df = pd.concat([df, a_sliding_df], axis=1)
    
#     df = pd.concat([df, adc_info.reset_index().iloc[:, 1:6]], axis=1)
    
#     return df

In [ ]:
# %%writefile -a utils.py

# def postprocessing(pred_array, index, sigma_pred):
#     """Create a submission dataframe from its components
    
#     Parameters:
#     pred_array: ndarray of shape (n_samples, 283)
#     index: pandas.Index of length n_samples with name 'planet_id'
#     sigma_pred: float
    
#     Return value:
#     df: DataFrame of shape (n_samples, 566) with planet_id as index
#     """
#     return pd.concat([pd.DataFrame(pred_array.clip(0, None), index=index, columns=wavelengths.columns),
#                       pd.DataFrame(sigma_pred, index=index, columns=[f"sigma_{i}" for i in range(1, 284)])],
#                      axis=1)

# class ParticipantVisibleError(Exception):
#     pass

# def competition_score(
#         solution: pd.DataFrame,
#         submission: pd.DataFrame,
#         naive_mean: float,
#         naive_sigma: float,
#         sigma_true: float,
#         row_id_column_name='planet_id',
#     ) -> float:
#     '''
#     This is a Gaussian Log Likelihood based metric. For a submission, which contains the predicted mean (x_hat) and variance (x_hat_std),
#     we calculate the Gaussian Log-likelihood (GLL) value to the provided ground truth (x). We treat each pair of x_hat,
#     x_hat_std as a 1D gaussian, meaning there will be 283 1D gaussian distributions, hence 283 values for each test spectrum,
#     the GLL value for one spectrum is the sum of all of them.

#     Inputs:
#         - solution: Ground Truth spectra (from test set)
#             - shape: (nsamples, n_wavelengths)
#         - submission: Predicted spectra and errors (from participants)
#             - shape: (nsamples, n_wavelengths*2)
#         naive_mean: (float) mean from the train set.
#         naive_sigma: (float) standard deviation from the train set.
#         sigma_true: (float) essentially sets the scale of the outputs.
#     '''

#     del solution[row_id_column_name]
#     del submission[row_id_column_name]

#     if submission.min().min() < 0:
#         raise ParticipantVisibleError('Negative values in the submission')
#     for col in submission.columns:
#         if not pd.api.types.is_numeric_dtype(submission[col]):
#             raise ParticipantVisibleError(f'Submission column {col} must be a number')

#     n_wavelengths = len(solution.columns)
#     if len(submission.columns) != n_wavelengths*2:
#         raise ParticipantVisibleError('Wrong number of columns in the submission')

#     y_pred = submission.iloc[:, :n_wavelengths].values
#     # Set a non-zero minimum sigma pred to prevent division by zero errors.
#     sigma_pred = np.clip(submission.iloc[:, n_wavelengths:].values, a_min=10**-15, a_max=None)
#     y_true = solution.values

#     GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred))
#     GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)))
#     GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=naive_mean * np.ones_like(y_true), scale=naive_sigma * np.ones_like(y_true)))

#     submit_score = (GLL_pred - GLL_mean)/(GLL_true - GLL_mean)
#     return float(np.clip(submit_score, 0.0, 1.0))

In [ ]:
# exec(open('utils.py', 'r').read())

### Load Data

In [ ]:
# %%time
# if os.path.exists("/kaggle/input/adc24-intro-training/f_raw_train.pickle"):
#     f_raw_train = np.load('/kaggle/input/adc24-intro-training/f_raw_train.pickle', allow_pickle=True)
# else:
#     f_raw_train = read_and_preprocess('train', train_labels.index, 'FGS1')
#     with open('f_raw_train.pickle', 'wb') as f:
#         pickle.dump(f_raw_train, f)

In [ ]:
# %%time
# if os.path.exists("/kaggle/input/adc24-intro-training/a_raw_train.pickle"):
#     a_raw_train = np.load('/kaggle/input/adc24-intro-training/a_raw_train.pickle', allow_pickle=True)
# else:
#     a_raw_train = read_and_preprocess('train', train_labels.index)
#     with open('a_raw_train.pickle', 'wb') as f:
#         pickle.dump(a_raw_train, f)

### Feature Engineering

In [ ]:
# %%time
# train = feature_engineering(f_raw_train, a_raw_train, train_adc_info)

In [ ]:
# train.head()

In [ ]:
# train = train.iloc[:,:-1]

### Data Plot

In [ ]:
# plt.figure(figsize=(6, 2))
# plt.plot(f_raw_train.mean(axis=0))
# for time_step in [20500, 23500, 44000, 47000]:
#     plt.axvline(time_step, color='gray')
# plt.xlabel('time step')
# plt.title('FGS1: Overall mean')
# plt.show()

# plt.figure(figsize=(6, 2))
# plt.plot(a_raw_train.mean(axis=0))
# for time_step in [20500, 23500, 44000, 47000]:
#     plt.axvline(time_step * 11250 // 135000, color='gray')
# plt.xlabel('time step')
# plt.title('AIRS-CH0: Overall mean')
# plt.show()

In [ ]:
# color_array = np.array(plt.rcParams['axes.prop_cycle'].by_key()['color'])
# plt.scatter(train.a_relative_reduction, train_labels.wl_1, s=15, alpha=0.5,
#             c=color_array[train_adc_info.star])
# plt.xlabel('relative signal reduction when planet is in front')
# plt.ylabel('target')
# plt.title('Correlation between relative signal reduction and target')
# # plt.gca().set_aspect('equal')
# points = [plt.Line2D([0], [0], label=f'star {i}', marker='o', markersize=3,
#          markeredgecolor=color_array[i], markerfacecolor=color_array[i], linestyle='') for i in range(2)]

# plt.legend(handles=points)
# plt.show()

### Model
#### Rigde Model

In [ ]:
# model = Ridge(alpha=1e-12)

# oof_pred = cross_val_predict(model, train, train_labels)

# print(f"# R2 score: {r2_score(train_labels, oof_pred):.4f}")
# sigma_pred = mean_squared_error(train_labels, oof_pred, squared=False)
# print(f"# Root mean squared error: {sigma_pred:.7f}")

In [ ]:
# oof_df = postprocessing(oof_pred, train_adc_info.index, sigma_pred)
# display(oof_df)

# gll_score = competition_score(train_labels.copy().reset_index(),
#                               oof_df.copy().reset_index(),
#                               naive_mean=train_labels.values.mean(),
#                               naive_sigma=train_labels.values.std(),
#                               sigma_true=0.000003)
# print(f"# Estimated competition score: {gll_score:.4f}")

In [ ]:
# model.fit(train, train_labels)
# with open('model.pickle', 'wb') as f:
#     pickle.dump(model, f)
# with open('sigma_pred.pickle', 'wb') as f:
#     pickle.dump(sigma_pred, f)

### Inference

In [ ]:
# # Load the data
# test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
#                            index_col='planet_id')
# sample_submission = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv',
#                                 index_col='planet_id')
# f_raw_test = read_and_preprocess('test', sample_submission.index, 'FGS1')
# a_raw_test = read_and_preprocess('test', sample_submission.index)
# test = feature_engineering(f_raw_test, a_raw_test, test_adc_info)
# test = test.iloc[: , :-1]
# # Load the model
# with open('model.pickle', 'rb') as f:
#     model = pickle.load(f)
# with open('sigma_pred.pickle', 'rb') as f:
#     sigma_pred = pickle.load(f)

# # Predict
# test_pred = model.predict(test)

# # Package into submission file
# sub_df = sub_df = postprocessing(test_pred,
#                         test_adc_info.index,
#                         sigma_pred=np.tile(np.where(test_adc_info[['star']] <= 1, 0.0001555, 0.00085), (1, 283)))
# display(sub_df)
# sub_df.to_csv('submission_5.csv')

## [ariel_only_correlation | param upd[LB.517]](https://www.kaggle.com/code/hideyukizushi/ariel-only-correlation-param-upd-lb-517)
### [yukiZ](https://www.kaggle.com/hideyukizushi)

### ℹ️ **Info**
* **forked original great work kernels**
    * https://www.kaggle.com/code/sergeifironov/ariel-only-correlation

* **2024/09/08 My Changed**
    * scipy minimize() param & other params update

---
---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
import itertools
from scipy.optimize import minimize
from functools import partial
import random, os
from astropy.stats import sigma_clip

In [ ]:
test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
                           index_col='planet_id')
axis_info = pd.read_parquet('/kaggle/input/ariel-data-challenge-2024/axis_info.parquet')

In [ ]:
def apply_linear_corr(linear_corr,clean_signal):
    linear_corr = np.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = np.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal

def clean_dark(signal, dark, dt):
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    signal -= dark* dt[:, np.newaxis, np.newaxis]
    return signal

def preproc(dataset, adc_info, sensor, binning = 15):
    cut_inf, cut_sup = 39, 321
    sensor_sizes_dict = {"AIRS-CH0":[[11250, 32, 356], [1, 32, cut_sup-cut_inf]], "FGS1":[[135000, 32, 32], [1, 32, 32]]}
    binned_dict = {"AIRS-CH0":[11250 // binning // 2, 282], "FGS1":[135000 // binning // 2]}
    linear_corr_dict = {"AIRS-CH0":(6, 32, 356), "FGS1":(6, 32, 32)}
    planet_ids = adc_info.index
    
    feats = []
    for i, planet_id in tqdm(list(enumerate(planet_ids))):
        signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/{planet_id}/{sensor}_signal.parquet').to_numpy()
        dark_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dark.parquet', engine='pyarrow').to_numpy()
        dead_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dead.parquet', engine='pyarrow').to_numpy()
        flat_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/flat.parquet', engine='pyarrow').to_numpy()
        linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/linear_corr.parquet').values.astype(np.float64).reshape(linear_corr_dict[sensor])

        signal = signal.reshape(sensor_sizes_dict[sensor][0]) 
        gain = adc_info[f'{sensor}_adc_gain'].values[i]
        offset = adc_info[f'{sensor}_adc_offset'].values[i]
        signal = signal / gain + offset
        
        hot = sigma_clip(
            dark_frame, sigma=8, maxiters=5
        ).mask
        
        if sensor != "FGS1":
            signal = signal[:, :, cut_inf:cut_sup] #11250 * 32 * 282
            #dt = axis_info['AIRS-CH0-integration_time'].dropna().values
            dt = np.ones(len(signal))*0.1 
            dt[1::2] += 4.5 #@bilzard idea
            linear_corr = linear_corr[:, :, cut_inf:cut_sup]
            dark_frame = dark_frame[:, cut_inf:cut_sup]
            dead_frame = dead_frame[:, cut_inf:cut_sup]
            flat_frame = flat_frame[:, cut_inf:cut_sup]
            hot = hot[:, cut_inf:cut_sup]
        else:
            dt = np.ones(len(signal))*0.1
            dt[1::2] += 0.1
            
        signal = signal.clip(0) #@graySnow idea
        linear_corr_signal = apply_linear_corr(linear_corr, signal)
        signal = clean_dark(linear_corr_signal, dark_frame, dt)
        
        flat = flat_frame.reshape(sensor_sizes_dict[sensor][1])
        flat[dead_frame.reshape(sensor_sizes_dict[sensor][1])] = np.nan
        flat[hot.reshape(sensor_sizes_dict[sensor][1])] = np.nan
        signal = signal / flat
        
        if sensor == "FGS1":
            signal = signal.reshape((sensor_sizes_dict[sensor][0][0], sensor_sizes_dict[sensor][0][1]*sensor_sizes_dict[sensor][0][2]))
        
        mean_signal = np.nanmean(signal, axis=1) # mean over the 32*32(FGS1) or 32(CH0) pixels
        cds_signal = (mean_signal[1::2] - mean_signal[0::2])
        
        binned = np.zeros((binned_dict[sensor]))
        for j in range(cds_signal.shape[0] // binning):
            binned[j] = cds_signal[j*binning:j*binning+binning].mean(axis=0)
                   
        if sensor == "FGS1":
            binned = binned.reshape((binned.shape[0],1))
            
        feats.append(binned)
        
    return np.stack(feats)
    
pre_train = np.concatenate([preproc('test', test_adc_info, "FGS1", 30*12), preproc('test', test_adc_info, "AIRS-CH0", 30)], axis=2)

### fit polynoms for each sample

In [ ]:
def phase_detector(signal):
    phase1, phase2 = None, None
    best_drop = 0
    for i in range(50//2,150//2):        
        t1 = signal[i:i+20//2].max() - signal[i:i+20//2].min()
        if t1 > best_drop:
            phase1 = i+(20+5)//2
            best_drop = t1
    
    best_drop = 0
    for i in range(200//2,250//2):
        t1 = signal[i:i+20//2].max() - signal[i:i+20//2].min()
        if t1 > best_drop:
            phase2 = i-5//2
            best_drop = t1
    
    return phase1, phase2

def try_s(signal, p1, p2, deg, s):
    out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
    x, y = out, signal[out].tolist()
    x = x + list(range(p1,p2))

    y = y + (signal[p1:p2] * (1 + s[0])).tolist()
    z = np.polyfit(x, y, deg)
    p = np.poly1d(z)
    q = np.abs(p(x) - y).mean()

    if s < 1e-4:
        return q + 1e3

    return q
    
def calibrate_signal(signal):
    p1,p2 = phase_detector(signal)

    best_deg, best_score = 1, 1e12
    for deg in range(1, 6):
        f = partial(try_s, signal, p1, p2, deg)
        r = minimize(f, [0.001], method = 'Nelder-Mead')
        s = r.x[0]

        out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
        x, y = out, signal[out].tolist()
        x = x + list(range(p1,p2))
        y = y + (signal[p1:p2] * (1 + s)).tolist()
    
        z = np.polyfit(x, y, deg)
        p = np.poly1d(z)
        q = np.abs(p(x) - y).mean()
        
        if q < best_score:
            best_score = q
            best_deg = deg
        
        print(deg, q)
            
    z = np.polyfit(x, y, best_deg)
    p = np.poly1d(z)

    return s, x, y, p(x)

def calibrate_train(signal):
    p1,p2 = phase_detector(signal)
    
    best_deg, best_score = 1, 1e12
    for deg in range(1, 6):
        f = partial(try_s, signal, p1, p2, deg)
        r = minimize(f, [0.0001], method = 'Nelder-Mead')
        s = r.x[0]

        out = list(range(p1-30)) + list(range(p2+30,signal.shape[0]))
        x, y = out, signal[out].tolist()
        x = x + list(range(p1,p2))
        y = y + (signal[p1:p2] * (1 + s)).tolist()
    
        z = np.polyfit(x, y, deg)
        p = np.poly1d(z)
        q = np.abs(p(x) - y).mean()
        
        if q < best_score:
            best_score = q
            best_deg = deg
            
    z = np.polyfit(x, y, best_deg)
    p = np.poly1d(z)
    
    return s, p(np.arange(signal.shape[0])), p1, p2


train = pre_train.copy()
all_s = []
for i in range(len(test_adc_info)):
    signal = train[i,:,1:].mean(axis=1)
    s, p, p1, p2 = calibrate_train(pre_train[i,:,1:].mean(axis=1))
    all_s.append(s)
        
#copy answer 283 times because we predict mean value
train_s = np.repeat(np.array(all_s), 283).reshape((len(all_s), 283))        
train_sigma = np.ones_like(train_s) * 0.000176

Probably we can accurately estimate sigma from train

In [ ]:
n = 0
s, x, y, y_new = calibrate_signal(pre_train[n,:,1:].mean(axis=1))
plt.scatter(x,y)
plt.scatter(x,y_new)

I call the orange line "starline". This is probably what we would see if the planet weren't in the way.

### Making submission

In [ ]:
ss = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv')

preds = train_s.clip(0)
sigmas = train_sigma
submission = pd.DataFrame(np.concatenate([preds,sigmas], axis=1), columns=ss.columns[1:])
submission.index = test_adc_info.index
submission.to_csv('submission_7.csv')

## [Ariel Data Challenge 2024](https://www.kaggle.com/code/xiaocao123/ariel-data-challenge-2024)
### [qianc](https://www.kaggle.com/xiaocao123)

This notebook is an update of https://www.kaggle.com/code/sergeifironov/ariel-only-correlation
from Sergei Fironov

Updates :
- keep 10:22 pixels from the 32 (the image are well centred)
- Use the derivative for the determination of the beginning and end of the signal during eclipse (idea from Reza R. Choubeh)
- 'Simplification' of the code for minimize
- Degree of polyfit <= 4
- Predictions of test after training Ridge regression with the modelization results (targets predictions with modelization) and the True targets. 

### Librairies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import joblib

from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
import itertools

from scipy.optimize import minimize
from scipy import optimize

from astropy.stats import sigma_clip

In [ ]:
dataset = 'test'
adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/'+f'{dataset}_adc_info.csv',index_col='planet_id')
axis_info = pd.read_parquet('/kaggle/input/ariel-data-challenge-2024/axis_info.parquet')

### Calibration

In [ ]:
def apply_linear_corr(linear_corr,clean_signal):
    linear_corr = np.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = np.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal

def clean_dark(signal, dark, dt):
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    signal -= dark* dt[:, np.newaxis, np.newaxis]
    return signal

def preproc(dataset, adc_info, sensor, binning = 15):
    cut_inf, cut_sup = 39, 321
    sensor_sizes_dict = {"AIRS-CH0":[[11250, 32, 356], [1, 32, cut_sup-cut_inf]], "FGS1":[[135000, 32, 32], [1, 32, 32]]}
    binned_dict = {"AIRS-CH0":[11250 // binning // 2, 282], "FGS1":[135000 // binning // 2]}
    linear_corr_dict = {"AIRS-CH0":(6, 32, 356), "FGS1":(6, 32, 32)}
    planet_ids = adc_info.index
    
    feats = []
    for i, planet_id in tqdm(list(enumerate(planet_ids))):
        signal = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/{planet_id}/{sensor}_signal.parquet').to_numpy()
        dark_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dark.parquet', engine='pyarrow').to_numpy()
        dead_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/dead.parquet', engine='pyarrow').to_numpy()
        flat_frame = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/flat.parquet', engine='pyarrow').to_numpy()
        linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration/linear_corr.parquet').values.astype(np.float64).reshape(linear_corr_dict[sensor])

        signal = signal.reshape(sensor_sizes_dict[sensor][0]) 
        gain = adc_info[f'{sensor}_adc_gain'].values[i]
        offset = adc_info[f'{sensor}_adc_offset'].values[i]
        signal = signal / gain + offset
        
        hot = sigma_clip(
            dark_frame, sigma=5, maxiters=5
        ).mask
        
        if sensor != "FGS1":
            signal = signal[:, :, cut_inf:cut_sup] 
            dt = np.ones(len(signal))*0.1 
            dt[1::2] += 4.5 #@bilzard idea
            linear_corr = linear_corr[:, :, cut_inf:cut_sup]
            dark_frame = dark_frame[:, cut_inf:cut_sup]
            dead_frame = dead_frame[:, cut_inf:cut_sup]
            flat_frame = flat_frame[:, cut_inf:cut_sup]
            hot = hot[:, cut_inf:cut_sup]
        else:
            dt = np.ones(len(signal))*0.1
            dt[1::2] += 0.1
            
        signal = signal.clip(0) #@graySnow idea
        linear_corr_signal = apply_linear_corr(linear_corr, signal)
        signal = clean_dark(linear_corr_signal, dark_frame, dt)
        
        flat = flat_frame.reshape(sensor_sizes_dict[sensor][1])
        flat[dead_frame.reshape(sensor_sizes_dict[sensor][1])] = np.nan
        flat[hot.reshape(sensor_sizes_dict[sensor][1])] = np.nan
        signal = signal / flat
        
        
        if sensor == "FGS1":
            signal = signal[:,10:22,10:22] # **** updates ****
            signal = signal.reshape(sensor_sizes_dict[sensor][0][0],144) # # **** updates ****

        if sensor != "FGS1":
            signal = signal[:,10:22,:] # **** updates ****

        mean_signal = np.nanmean(signal, axis=1) 
        cds_signal = (mean_signal[1::2] - mean_signal[0::2])
        
        binned = np.zeros((binned_dict[sensor]))
        for j in range(cds_signal.shape[0] // binning):
            binned[j] = cds_signal[j*binning:j*binning+binning].mean(axis=0) 
                   
        if sensor == "FGS1":
            binned = binned.reshape((binned.shape[0],1))
        
        feats.append(binned)
        
    return np.stack(feats)
    
pre_train = np.concatenate([preproc(f'{dataset}', adc_info, "FGS1", 30*12), preproc(f'{dataset}', adc_info, "AIRS-CH0", 30)], axis=2)

### Modelization

In [ ]:
def phase_detector(signal):
    
    MIN = np.argmin(signal[30:140])+30
    signal1 = signal[:MIN ]
    signal2 = signal[MIN :]

    first_derivative1 = np.gradient(signal1)
    first_derivative1 /= first_derivative1.max()
    first_derivative2 = np.gradient(signal2)
    first_derivative2 /= first_derivative2.max()

    phase1 = np.argmin(first_derivative1)
    phase2 = np.argmax(first_derivative2) + MIN

    return phase1, phase2
    
def objective(s):
    
    best_q = 1e10
    for i in range(4) :
        delta = 2
        x = list(range(signal.shape[0]-delta*4))
        y = signal[:p1-delta].tolist() + (signal[p1+delta:p2 - delta] * (1 + s)).tolist() + signal[p2+delta:].tolist()
        
        z = np.polyfit(x, y, deg=i)
        p = np.poly1d(z)
        q = np.abs(p(x) - y).mean()
    
    if q < best_q :
        best_q = q
    
    return q


all_s = []
for i in tqdm(range(len(adc_info))):
    
    signal = pre_train[i,:,1:].mean(axis=1)
    p1,p2 = phase_detector(signal)
 
    r = minimize(
                objective,
                [0.0001],
                method= 'Nelder-Mead'
                  )
    s = r.x[0]
    all_s.append(s)
    
all_s = np.repeat(np.array(all_s), 283).reshape((len(all_s), 283))        

### Predictions with Ridge model

In [ ]:
# pd.DataFrame(all_s)

In [ ]:
# model = joblib.load("/kaggle/input/adc24-meta-model-ridge/model_ridge_10_22_delta2.joblib")
# pred = model.predict(all_s)
# pd.DataFrame(pred)
# import pickle
# with open('/kaggle/input/ad24-train-inf-ridge-addfe-lb-441/model.pickle', 'rb') as f:
#     model = pickle.load(f)
# pred = model.predict(all_s)
# pd.DataFrame(pred)

### Submission

In [ ]:
ss = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/sample_submission.csv')
sigma = np.ones_like(all_s) * 0.0001422 
pred = all_s.clip(0) 
submission = pd.DataFrame(np.concatenate([pred,sigma], axis=1), columns=ss.columns[1:])
submission.index = adc_info.index
submission.to_csv('submission_8.csv')
submission


## Ensemble of solutions

In [ ]:
# #### options

# - option 1 -> LB=0.515 work (1,2)
# - option 2 -> LB= ?.??? work (3,4,5) 
# - option 3 -> LB=0.000 work (1,2,3)         
# - option 4 -> LB= ?.??? work (4,5)
# - option 5 -> LB=0.514 work (1,2) + weight (0.15+0.85)
# - option 7 -> LB=0.515 work (1,2) + weight (0.85+0.15)
# - option 8 -> LB=0.515 work (1,2) + weight (0.07+0.93)
# - option 9 -> LB=0.517 work (2,7)
# - option 10 -> LB=0.516 work (1,2,7)
# - option 11 -> LB=0.517 work (2,7) + weight (0.10+0.90)
# - option 12 -> LB=0.517 work (2,7) + weight (0.05+0.95)
# - option 14 -> LB=0.544 work (7,8) + weight (0.005+0.995)
# - option 15 -> LB=0.544 work (7,8) + weight (0.008+0.992)
# - option 16 -> LB=0.544 work (7,8) + weight (0.011+0.989)
# - option 17 -> LB=0.544 work (7,8) + weight (0.015+0.985)
# - option 18 -> LB=**0.544** work (7,8) + weight (0.02+0.98)
# - option 19 -> LB=0.544 work (7,8) + weight (0.05+0.95)
# - option 20 -> LB=0.544 work (7,8) + weight (0.07+0.93)

# option.14 < option.15 < option.16 < option.17 < **option.18**
# option.19 < option.20 < **option.18**

# best option:
# - option 18 -> LB=0.544 work (7,8) + weight (0.02+0.98), **Version 37** 

# current option: 
# - option 21 -> work (7,8) + weight (0.03+0.97)
# - option 22 -> work (7,8) + weight (0.04+0.96)

# next option: 
# - option 23 -> ?

In [ ]:
if LAUNCH_VARIANT == 'option 22':
    df_solution_x = pd.read_csv('submission_7.csv')
    df_solution_7 = pd.read_csv('submission_7.csv')
    df_solution_8 = pd.read_csv('submission_8.csv')
    df_solution_7 = df_solution_7.map(lambda x:x*0.04)
    df_solution_8 = df_solution_8.map(lambda x:x*0.96)
    df_temp = df_solution_7.add(df_solution_8)
    df_submission = df_temp.map(lambda x:x)
    df_submission['planet_id'] = df_solution_x['planet_id']
    
elif LAUNCH_VARIANT == 'option 21':
    df_solution_x = pd.read_csv('submission_7.csv')
    df_solution_7 = pd.read_csv('submission_7.csv')
    df_solution_8 = pd.read_csv('submission_8.csv')
    df_solution_7 = df_solution_7.map(lambda x:x*0.03)
    df_solution_8 = df_solution_8.map(lambda x:x*0.97)
    df_temp = df_solution_7.add(df_solution_8)
    df_submission = df_temp.map(lambda x:x)
    df_submission['planet_id'] = df_solution_x['planet_id']

In [ ]:
df_submission.to_csv('submission.csv', index=False, float_format='%.7f')
df_submission

In [ ]:
#                                          arhiv options
# if LAUNCH_VARIANT == 'option 20':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.07)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.93)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
    
# elif LAUNCH_VARIANT == 'option 19':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.05)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.95)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 18':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.020)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.980)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
    
# elif LAUNCH_VARIANT == 'option 17':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.015)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.985)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 16':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.011)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.989)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 15':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.008)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.992)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 14':
#     df_solution_x = pd.read_csv('submission_7.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_8 = pd.read_csv('submission_8.csv')
#     df_solution_7 = df_solution_7.map(lambda x:x*0.005)
#     df_solution_8 = df_solution_8.map(lambda x:x*0.995)
#     df_temp = df_solution_7.add(df_solution_8)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 12':
#     df_solution_x = pd.read_csv('submission_2.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_2 = df_solution_2.map(lambda x:x*0.05)
#     df_solution_7 = df_solution_7.map(lambda x:x*0.95)
#     display(df_solution_2, df_solution_7)
#     df_temp = df_solution_2.add(df_solution_7)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 11':
#     df_solution_x = pd.read_csv('submission_2.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_7 = pd.read_csv('submission_7.csv')
#     df_solution_2 = df_solution_2.map(lambda x:x*0.10)
#     df_solution_7 = df_solution_7.map(lambda x:x*0.90)
#     display(df_solution_2, df_solution_7)
#     df_temp = df_solution_2.add(df_solution_7)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 10':
#     df_solution_1 = pd.read_csv('submission_1.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_6 = pd.read_csv('submission_6.csv')
#     display(df_solution_1, df_solution_2, df_solution_6)
#     df_temp = df_solution_1.add(df_solution_2).add(df_solution_6)
#     df_submission = df_temp.map(lambda x: x/3)
#     df_submission['planet_id'] = df_solution_1['planet_id']
#
# elif LAUNCH_VARIANT == 'option 9':
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_6 = pd.read_csv('submission_6.csv')
#     display(df_solution_2, df_solution_6)
#     df_temp = df_solution_2.add(df_solution_6)
#     df_submission = df_temp.map(lambda x: x/2)
#     df_submission['planet_id'] = df_solution_2['planet_id']
# 
# elif LAUNCH_VARIANT == 'option 8':
#     df_solution_x = pd.read_csv('submission_1.csv')
#     df_solution_1 = pd.read_csv('submission_1.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_1 = df_solution_1.map(lambda x: x*0.07)
#     df_solution_2 = df_solution_2.map(lambda x: x*0.93)
#     display(df_solution_1, df_solution_2)
#     df_temp = df_solution_1.add(df_solution_2)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 7':
#     df_solution_x = pd.read_csv('submission_1.csv')
#     df_solution_1 = pd.read_csv('submission_1.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_1 = df_solution_1.map(lambda x: x*0.85)
#     df_solution_2 = df_solution_2.map(lambda x: x*0.15)
#     display(df_solution_1, df_solution_2)
#     df_temp = df_solution_1.add(df_solution_2)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 5':
#     df_solution_x = pd.read_csv('submission_1.csv')
#     df_solution_1 = pd.read_csv('submission_1.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_1 = df_solution_1.map(lambda x: x*0.15)
#     df_solution_2 = df_solution_2.map(lambda x: x*0.85)
#     display(df_solution_1, df_solution_2)
#     df_temp = df_solution_1.add(df_solution_2)
#     df_submission = df_temp.map(lambda x:x)
#     df_submission['planet_id'] = df_solution_x['planet_id']
#
# elif LAUNCH_VARIANT == 'option 4':
#     df_solution_4 = pd.read_csv('submission_4.csv')
#     df_solution_5 = pd.read_csv('submission_5.csv')
#     display(df_solution_4, df_solution_5)
#     df_temp = df_solution_4.add(df_solution_5)
#     df_submission = df_temp.map(lambda x: x/2)
#     df_submission['planet_id'] = df_solution_1['planet_id']
# 
# elif LAUNCH_VARIANT == 'option 3':
#     df_solution_1 = pd.read_csv('submission_1.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     df_solution_3 = pd.read_csv('submission_3.csv')
#     display(df_solution_1, df_solution_2, df_solution_3)
#     df_temp = df_solution_1.add(df_solution_2).add(df_solution_3)
#     df_submission = df_temp.map(lambda x: x/3)
#     df_submission['planet_id'] = df_solution_1['planet_id']
#
# elif LAUNCH_VARIANT == 'option 2':
#     df_solution_3 = pd.read_csv('submission_3.csv')
#     df_solution_4 = pd.read_csv('submission_4.csv')
#     df_solution_5 = pd.read_csv('submission_5.csv')
#     display(df_solution_3, df_solution_4, df_solution_5)
#     df_temp = df_solution_3.add(df_solution_4).add(df_solution_5)
#     df_submission = df_temp.map(lambda x: x/3)
#     df_submission['planet_id'] = df_solution_3['planet_id']
#      
# elif LAUNCH_VARIANT == 'option 1':
#     df_solution_1 = pd.read_csv('submission_1.csv')
#     df_solution_2 = pd.read_csv('submission_2.csv')
#     display(df_solution_1, df_solution_2)
#     df_temp = df_solution_1.add(df_solution_2)
#     df_submission = df_temp.map(lambda x: x/2)
#     df_submission['planet_id'] = df_solution_1['planet_id']